In [1]:
import numpy as np
import pykep as pk
import sys
sys.path.append("../..")
import paseos
from paseos import ActorBuilder, SpacecraftActor
from paseos.utils.reference_frame_transfer import eci_to_rpy, rpy_to_body

In [2]:
def flux_density_vector(actor, frame="eci"):
        """Function to calculate the local magnetic field flux density vector (B) at the actor's location.
        B vector is calculated multiple times. Use of this function for code clarity.

        Args:
            actor (SpacecraftActor): Spacecraft actor
            frame (string): B expressed in which frame (actor body frame or Earth-centered inertial frame)

        Returns: B vector (np.ndarray)
        """
        # get Earth B vector at specific timestep
        # Earth magnetic dipole moment:
        m_earth = actor.central_body.magnetic_dipole_moment(actor.local_time)

        # parameters to calculate local B vector:
        actor_position = np.array(actor.get_position(actor.local_time))
        r = np.linalg.norm(actor_position)
        r_hat = actor_position / r

        # local B vector:
        B = 1e-7 * (3 * np.dot(m_earth, r_hat) * r_hat - m_earth) / (r**3)
        if frame == "eci":
            # return B vector in ECI frame
            return B
        elif frame == "body":
            # transform B vector to the actor body frame and return
            actor_velocity = np.array(actor.get_position_velocity(actor.local_time)[1])
            actor_attitude = np.array(actor.attitude_in_rad)
            return rpy_to_body(
                eci_to_rpy(B, actor_position, actor_velocity), actor_attitude
            )

In [3]:
# Define central body
earth = pk.planet.jpl_lp("earth")

# Define spacecraft actors
# magnetic:
sat1 = ActorBuilder.get_actor_scaffold("sat1", SpacecraftActor, pk.epoch(0))
# non-magnetic:
sat2 = ActorBuilder.get_actor_scaffold("sat2", SpacecraftActor, pk.epoch(0))

# Geostationary orbital parameters:
r = 6371000 + 35786000  # radius [km]
v = 3074.66  # velocity [m/s]

# To have a more symmetric case, let the actors be on same longitude as Earth magnetic dipole vector
longitude = -71.6 * np.pi / 180

# Set orbits:
ActorBuilder.set_orbit(
    sat1,
    position=[
        r * np.cos(np.pi / 2 + longitude),
        r * np.sin(np.pi / 2 + longitude),
        0,
    ],
    velocity=[
        -v * np.sin(np.pi / 2 + longitude),
        v * np.cos(np.pi / 2 + longitude),
        0,
    ],
    epoch=pk.epoch(0),
    central_body=earth,
)
ActorBuilder.set_orbit(
    sat2,
    position=[
        r * np.cos(np.pi / 2 + longitude),
        r * np.sin(np.pi / 2 + longitude),
        0,
    ],
    velocity=[
        -v * np.sin(np.pi / 2 + longitude),
        v * np.cos(np.pi / 2 + longitude),
        0,
    ],
    epoch=pk.epoch(0),
    central_body=earth,
)

# Set geometric model
ActorBuilder.set_spacecraft_body_model(sat1, mass=100)
ActorBuilder.set_spacecraft_body_model(sat2, mass=100)

In [4]:
B0 = np.array([-3.18159529e-09, 1.02244882e-07, -3.72362170e-08])

B_direction = B0 / np.linalg.norm(B0)

# Define a very large dipole moment for magnetic actor to compensate for the low magnetic field at GEO orbit
m_body = 500  # Am²
actor_dipole = np.ndarray.tolist(B_direction * m_body)
initial_pointing_vector_body = np.ndarray.tolist(B_direction)

# Set attitude models
ActorBuilder.set_attitude_model(
    sat1,
    actor_initial_angular_velocity=[0.0, 0.0, 0.0],
    actor_pointing_vector_body=initial_pointing_vector_body,
    actor_initial_attitude_in_rad=[0.0, 0.0, 0.0],
    actor_residual_magnetic_field=actor_dipole,  # magnetic
)

In [5]:
ActorBuilder.set_attitude_disturbances(sat1, magnetic=True)

In [6]:
sat1.attitude_disturbances

['magnetic']

In [7]:
# Initial pointing vector in Earth inertial frame
initial_pointing_vector_eci = np.array(sat1.pointing_vector)
# Start simulation
sim = paseos.init_sim(sat1)
sim.add_known_actor(sat2)

# Check if B0, used to define the actors' magnetic field orientations is the initial B vector orientation in sim.
assert np.all(np.isclose(B0, flux_density_vector(sat1, "body")))

# The magnetic actor will oscillate slightly, following the Earth's magnetic field lines. (check for multiple steps)
# The actor's magnetic field will not stay perfectly aligned with the Earth's field but needs to stay close.
for i in range(20):
    sim.advance_time(200, 0)

    # Get the magnetic flux density vector:
    B = flux_density_vector(sat1)

    # B vector direction:
    B_direction = B / np.linalg.norm(B)

    # Angle between the B vector and the actor's magnetic dipole vector (which is in the pointing vector direction):
    angle = np.arccos(np.dot(B_direction, sat1.pointing_vector)) * 180 / np.pi
    print(angle)
    # Check if the magnetic

AttributeError: 'SpacecraftActor' object has no attribute 'get_disturbances'

In [ ]:
angle